In [1]:
# From https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [2]:
!pip install tensorflow

In [3]:
!pip install tensorflow-addons

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train_knn = np.load("add_feat_train.npy")
test_knn = np.load("add_feat_test.npy")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_knn = scaler.fit_transform(train_knn)
test_knn = scaler.transform(test_knn)

In [7]:
train = pd.concat([train, 
                   pd.DataFrame(train_knn, columns = ['knn_1', 'knn_2', 'knn_3', 'knn_4', 'knn_5', 'knn_6', 'knn_7', 'knn_8', 'knn_9'])], axis = 1)
test = pd.concat([test, 
                   pd.DataFrame(test_knn, columns = ['knn_1', 'knn_2', 'knn_3', 'knn_4', 'knn_5', 'knn_6', 'knn_7', 'knn_8', 'knn_9'])], axis = 1,)

In [8]:
print(len(train))
non_id_columns = train.columns[train.columns != 'id']
train = train.drop_duplicates(subset=non_id_columns)
print(len(train))

200000
199964


In [9]:
targets = pd.get_dummies(train['target'])
temp_target = train['target']
train = train.drop('target', 1)
train['target'] = temp_target
X_dim = np.shape(train)[1] - 2 # id and target
print(X_dim)

print(train.columns)

84
Index(['id', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
       'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',
       'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54',
       'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59',
       'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64',


In [10]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [11]:
def conv_model():

    conv_inputs = layers.Input(shape = (X_dim))
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'relu',
                kernel_initializer = "lecun_normal"))(output)
    
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [12]:
import datetime;
import time;

def do_run(run_number):
    print(datetime.datetime.now())
    start = time.time()

    oof_NN_a = np.zeros((train.shape[0],9))
    pred_NN_a_train = np.zeros((train.shape[0],9))
    pred_NN_a_test = np.zeros((test.shape[0],9))

        N_FOLDS = 25
        EPOCH = 60

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

    for fold, (tr_idx, ts_idx) in enumerate(skf.split(train,train.iloc[:,-1])):
        #print(f"\n ====== TRAINING FOLD {fold} =======")

        X_train = train.iloc[:,1:-1].iloc[tr_idx]
        y_train = targets.iloc[tr_idx]
        X_test = train.iloc[:,1:-1].iloc[ts_idx]
        y_test = targets.iloc[ts_idx]

        K.clear_session()

        #print("\n-----Convolution model Training----\n")
        model_conv = conv_model()

        model_conv.compile(loss='categorical_crossentropy', 
                                optimizer= 'adam', #keras.optimizers.Adam(learning_rate=2e-4), 
                                metrics=custom_metric)
        
        #K.set_value(model_conv.optimizer.learning_rate, 2e-4)
        model_conv.fit(X_train, y_train,
                  batch_size = 256, epochs = EPOCH,
                  validation_data=(X_test, y_test),
                  callbacks=[es, plateau],
                  verbose = 0)

        pred_a = model_conv.predict(X_test) 
        oof_NN_a[ts_idx] += pred_a 
        score_NN_a = log_loss(y_test, pred_a)
        print(f"RUN {run_number} - FOLD {fold} - Score: {score_NN_a}")
        pred_NN_a_test += model_conv.predict(test.iloc[:,1:]) / N_FOLDS 
        
        train_no_target = train.drop('target', 1)
        pred_NN_a_train += model_conv.predict(train_no_target.iloc[:,1:]) / N_FOLDS 

    score_a = log_loss(targets, oof_NN_a)
    print(f"=== FINAL SCORE CONVOLUTION MODEL : {score_a} ===")
    print(datetime.datetime.now())
    minutes = (time.time() - start) / 60
    print('minutes', minutes, '\n')
    
    return pred_NN_a_test, pred_NN_a_train
    

In [13]:
n_runs = 5
test_result, train_result = do_run(1)

for i in range(2, n_runs+1):
    loop_test_result, loop_train_result = do_run(i)
    test_result = test_result + loop_test_result
    train_result = train_result + loop_train_result
    
test_result = test_result / n_runs
train_result = train_result / n_runs

2021-06-30 00:36:24.706116
RUN 1 - FOLD 0 - Score: 1.74633478170608
RUN 1 - FOLD 1 - Score: 1.7373416451037542
RUN 1 - FOLD 2 - Score: 1.7486232212522388
RUN 1 - FOLD 3 - Score: 1.7492268492313665
RUN 1 - FOLD 4 - Score: 1.7437724112711752
RUN 1 - FOLD 5 - Score: 1.747198517295398
RUN 1 - FOLD 6 - Score: 1.7504948992412555
RUN 1 - FOLD 7 - Score: 1.7436300259582296
RUN 1 - FOLD 8 - Score: 1.748569835371387
RUN 1 - FOLD 9 - Score: 1.7492865081104587
RUN 1 - FOLD 10 - Score: 1.7535311793259962
RUN 1 - FOLD 11 - Score: 1.7518279097770781
RUN 1 - FOLD 12 - Score: 1.7423465452987688
RUN 1 - FOLD 13 - Score: 1.7462676076491164
RUN 1 - FOLD 14 - Score: 1.7456961815632277
RUN 1 - FOLD 15 - Score: 1.7497278940466832
RUN 1 - FOLD 16 - Score: 1.7313252133383665
RUN 1 - FOLD 17 - Score: 1.7522887073622126
RUN 1 - FOLD 18 - Score: 1.749486725716138
RUN 1 - FOLD 19 - Score: 1.7401820575465825
RUN 1 - FOLD 20 - Score: 1.740031454964604
RUN 1 - FOLD 21 - Score: 1.7453640174759013
RUN 1 - FOLD 22 - Sco

In [16]:
abs(log_loss(train['target'], train_result))

1.7285641582995286

In [17]:
submission = pd.read_csv("./sample_submission.csv")
submission['Class_1']=test_result[:,0]
submission['Class_2']=test_result[:,1]
submission['Class_3']=test_result[:,2]
submission['Class_4']=test_result[:,3]
submission['Class_5']=test_result[:,4]
submission['Class_6']=test_result[:,5]
submission['Class_7']=test_result[:,6]
submission['Class_8']=test_result[:,7]
submission['Class_9']=test_result[:,8]

submission.to_csv("submission_cnn_ensemble_test.csv", index=False)